In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from sklearn.utils.multiclass import unique_labels

from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.utils import resample


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
#### read in the flattened dataframes with the labels for the 98 to 2015 data
MSFT_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_MSFT_98_15.csv")
AAPL_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AAPL_98_15.csv")
AMZN_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AMZN_98_15.csv")

In [3]:
###read in the flattened dataframes with the labels for 2015 to 2019
MSFT_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_MSFT_15_19.csv")
AAPL_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AAPL_15_19.csv")
AMZN_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AMZN_15_19.csv")



In [4]:
#vertically stack the MSFT and AAPL dataframes on top of eachother
df = pd.concat([MSFT_df_98_to_15_df, AAPL_df_98_to_15_df], axis=0)
#vertically stack the MSFT&AAPL df and the AMZN dataframe ontop of eachother
df = pd.concat([df,AMZN_df_98_to_15_df],axis = 0)

df = df.drop(['Unnamed: 0'], axis = 1)

y_train = df.Label
y_train = pd.DataFrame(y_train)
y_train
export_multiclass_labels = y_train.to_csv(r"y_train_multiclass_labels.csv")


#X_train = df.drop(['Label'], axis =1)


# df_upsampled = df.drop(['Label'], axis=1)

# df_upsampled

# # df

# X_train.shape



# #drop the dates column
# df = df.drop(['Unnamed: 0'], axis = 1)
# #store the labels as the y_train array object 
# y_train = df['Label']
# #drop the labels from the instances
# df = df.drop(['Label'], axis =1)
# #assign the remaining columns in the dataframe
# X_train = df


In [49]:
print (X_train.shape)
print (y_train.shape)

(912, 360)
(912,)


In [5]:
df_2 = pd.concat([MSFT_df_15_to_19_df, AAPL_df_15_to_19_df], axis=0)
df_2 = pd.concat([df_2,AMZN_df_15_to_19_df],axis = 0)
dates = df_2['Unnamed: 0']

df_2 = df_2.drop(['Unnamed: 0'], axis = 1)

#get labels for the prediction data
y_test = df_2['Label']

#drop the labelss
df_2 = df_2.drop(['Label'], axis=1)

#X_test to predict the values
X_test = df_2

In [7]:
y_test = pd.DataFrame(y_test)


In [8]:
export_ytest = y_test.to_csv("y_test_multiclass.csv")

In [50]:
#prev 50 and 150
es = EarlyStopping(monitor='val_acc', mode='max', verbose=0, patience=200)
mc = ModelCheckpoint('1dconv_feature_engineered_model_chronological_multiclass.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
cb_list = [es,mc]

# X_Train

In [56]:
#list which will become the array of shape 
list_1 = list()
#transform each row into a (30,12) array append to the list
for index, row in X_train.iterrows():
    arr = np.array(row).reshape(30,12)
    list_1.append(arr)

    #print(arr)
#     print (arr)
#     print (arr.shape)
#     list_1.append(arr)

arr1 = np.asarray(list_1)

# #arr1 is an array of all the arrays needed to utilise the 
X_train = arr1
    

X_train.shape



(912, 30, 12)

# X_test

In [10]:
list_2 = list()

for index, row in df_2.iterrows():
    arr_2 = np.array(row).reshape(30,12)
    list_2.append(arr_2)
    
arr2 = np.asarray(list_2)

X_test = arr2

X_test.shape

(150, 30, 12)

In [58]:
def make_timeseries_classifier(X_train, y_train, X_test, y_test):

    
    verbose, epochs, batch_size = 0, 10, 32
    
    #number of timesteps for each prediction is 30, number of features is 12, number of outputs is 1
    n_timesteps, n_features, n_outputs =1,360, 4
    
    model = Sequential()
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu',
    input_shape=(n_timesteps,n_features)))
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu')) 
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(4, activation='softmax')) 
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy']) # fit network
    
    return model

NameError: name 'X_train' is not defined

In [59]:
model_1 = make_timeseries_classifier(X_train, y_train, X_test, y_test)
verbose, epochs, batch_size = 0, 10, 32
model_1.summary()
print (model_1.inputs)

es = EarlyStopping(monitor='val_acc', mode='max', verbose=0, patience=75)
mc = ModelCheckpoint('1dconv_feature_engineered_model_chronological_multiclass.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
cb_list = [es,mc]





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 28, 64)            2368      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 26, 64)            12352     
_________________________________________________________________
dropout_3 (Dropout)          (None, 26, 64)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 832)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               83300     
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 404       
Total para

In [60]:
model_1.fit(X_train, y_train, batch_size=128,
          validation_split=0.10, epochs=400
            , verbose=1, callbacks = cb_list)

W0810 16:25:37.983129 4753233344 deprecation.py:323] From /Users/fahad/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 820 samples, validate on 92 samples
Epoch 1/400
820/820 [==============================] - 1s 750us/step - loss: 1.3969 - acc: 0.2512 - val_loss: 1.7257 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to 1dconv_feature_engineered_model_chronological_multiclass.h5
Epoch 2/400
820/820 [==============================] - 0s 88us/step - loss: 1.3569 - acc: 0.3549 - val_loss: 1.6204 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.00000
Epoch 3/400
820/820 [==============================] - 0s 86us/step - loss: 1.3391 - acc: 0.3902 - val_loss: 1.6333 - val_acc: 0.0000e+00

Epoch 00003: val_acc did not improve from 0.00000
Epoch 4/400
820/820 [==============================] - 0s 85us/step - loss: 1.3072 - acc: 0.4244 - val_loss: 1.6160 - val_acc: 0.0000e+00

Epoch 00004: val_acc did not improve from 0.00000
Epoch 5/400
820/820 [==============================] - 0s 88us/step - loss: 1.2717 - acc: 0.4463 - val_loss: 1.6480 - va

820/820 [==============================] - 0s 79us/step - loss: 0.2516 - acc: 0.9098 - val_loss: 0.7621 - val_acc: 0.8152

Epoch 00081: val_acc did not improve from 0.83696
Epoch 82/400
820/820 [==============================] - 0s 81us/step - loss: 0.2284 - acc: 0.9366 - val_loss: 0.6467 - val_acc: 0.8587

Epoch 00082: val_acc improved from 0.83696 to 0.85870, saving model to 1dconv_feature_engineered_model_chronological_multiclass.h5
Epoch 83/400
820/820 [==============================] - 0s 93us/step - loss: 0.2499 - acc: 0.9159 - val_loss: 0.6983 - val_acc: 0.8478

Epoch 00083: val_acc did not improve from 0.85870
Epoch 84/400
820/820 [==============================] - 0s 89us/step - loss: 0.2245 - acc: 0.9427 - val_loss: 0.7655 - val_acc: 0.8478

Epoch 00084: val_acc did not improve from 0.85870
Epoch 85/400
820/820 [==============================] - 0s 83us/step - loss: 0.2155 - acc: 0.9354 - val_loss: 0.8596 - val_acc: 0.8152

Epoch 00085: val_acc did not improve from 0.85870
Ep

820/820 [==============================] - 0s 78us/step - loss: 0.0613 - acc: 0.9854 - val_loss: 0.7581 - val_acc: 0.8804

Epoch 00167: val_acc did not improve from 0.88043
Epoch 168/400
820/820 [==============================] - 0s 77us/step - loss: 0.0515 - acc: 0.9878 - val_loss: 0.8257 - val_acc: 0.8804

Epoch 00168: val_acc did not improve from 0.88043
Epoch 169/400
820/820 [==============================] - 0s 75us/step - loss: 0.0448 - acc: 0.9927 - val_loss: 0.8259 - val_acc: 0.8804

Epoch 00169: val_acc did not improve from 0.88043
Epoch 170/400
820/820 [==============================] - 0s 77us/step - loss: 0.0532 - acc: 0.9854 - val_loss: 0.8704 - val_acc: 0.8696

Epoch 00170: val_acc did not improve from 0.88043
Epoch 171/400
820/820 [==============================] - 0s 76us/step - loss: 0.0464 - acc: 0.9878 - val_loss: 0.8230 - val_acc: 0.8804

Epoch 00171: val_acc did not improve from 0.88043
Epoch 172/400
820/820 [==============================] - 0s 74us/step - loss: 0

In [61]:
saved_model = load_model('1dconv_feature_engineered_model_chronological_multiclass.h5')



In [62]:
predictions = saved_model.predict_classes(X_test,batch_size = X_test.shape[0], verbose=1)

from sklearn.metrics import accuracy_score
print ("saved model acc: %s" % accuracy_score(y_test, predictions))
# accuracy: (tp + tn) / (p + n)

# precision tp / (tp + fp)
precision = precision_score(
    y_test,
    predictions,
    average=None)


recall = recall_score(y_test,
                      predictions, 
                     average=None)



150/150 [==============================] - 0s 508us/step
saved model acc: 0.4066666666666667
